In [2]:
import boto3
import os

In [5]:
# Connect to AWS
# Make sure you have the AWS credentials in your .secret file
with open("/home/dbogdoll/mcity_data_engine/.secret", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        os.environ[key] = value

s3 = boto3.client(
    "s3",
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID", None),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY", None),
)

In [12]:
def _process_aws_result(result):
    # Get list of folders from AWS response
    if "CommonPrefixes" in result:
        folders = [prefix["Prefix"] for prefix in result["CommonPrefixes"]]
        return folders
    else:
        return None

In [20]:
def list_files(bucket, prefix):
    files = []
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        if 'Contents' in page:
            for obj in page['Contents']:
                files.append(obj['Key'])
    
    return files

In [ ]:
bucket = "mcity-data-engine"
prefix = ""
all_files = list_files(bucket, prefix)

print(f"Found {len(all_files)} files in {bucket}/{prefix}")

download_path = "/media/dbogdoll/Datasets/data_engine_rolling_processed"
for file in all_files:
    file_path = os.path.join(download_path, file)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    s3.download_file(bucket, file, file_path)
    print(f"Downloaded {file} to {file_path}")

1/sip-data-stream2-delivery-stream-2-2024-03-07-17-36-56-73993be5-03fe-3cdd-baf2-1c46c6fc400e_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-03-07-18-13-55-83378470-fa8c-38a8-aca8-03b75f2b15fd_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-03-07-18-13-56-3b988640-6ec0-3513-bbbe-4129231a52b6_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-03-07-18-13-57-0dc57636-cadb-3fbc-8a97-75060fab0b21_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-03-07-18-14-02-80bb8ed5-f4c5-39fb-b68f-4ba535a64aa9_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-03-07-18-14-05-01a3dd5f-b826-3a49-9861-2833f9e46bf4_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-04-04-18-26-39-1f41670f-12e0-3191-bb7e-e0fbd5387423_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-04-04-18-39-24-6037ff71-05e9-32ed-96d6-71dbcf543f09_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-04-04-18-39-24-98c2d5a7-0dea-3c81-b58d-3bce6999d179_sampled_1Hz
1/sip-data-stream2-delivery-stream-2-2024-04-0